In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

In [2]:
class RTScraper:
    
    def __init__(
        self,
        rss_link:str,
    ):
        self.rss_link = rss_link
        return
    
    def get_latest_news(self)->pd.DataFrame:
        news = self.__get_news()
        news_data = {
            'title':[],
            'link':[],
            'publication_time':[],
            'text':[],
            
        }
        for piece_of_new in news:
            news_data['title'].append(piece_of_new.find('title').text)
            news_data['link'].append(piece_of_new.find('guid').text)
            news_data['publication_time'].append(self.__get_publication_time(piece_of_new))
            news_data['text'].append(self.__get_text(piece_of_new))
        return pd.DataFrame.from_dict(news_data)
            
    def __get_news(self):
        html_page = requests.get(self.rss_link).text
        html_page = BeautifulSoup(html_page, 'html.parser')
        news = html_page.findAll('item')
        return news
    
    def __get_publication_time(
        self,
        piece_of_new:bs4.element.Tag,
    )->datetime.datetime:
        time_format = '%d %b %Y %H:%M:%S %z'
        time = piece_of_new.find('pubdate').text
        time = time.split(',')[1].strip()
        time = datetime.datetime.strptime(time, time_format)
        return time
    
    def __get_text(
        self,
        piece_of_new:bs4.element.Tag,
    )->str:
        """Надо дописать, оч кривая структура"""
        description = piece_of_new.find('description')
        if 'img alt' in description.text:
            raw_text = description.text.split('<br')[0].split('>')[-1]
        else:
            raw_text = description.text.split('<br')[0]
        return ''.join([char for char in raw_text if char.isalnum() or char == ' ']).strip()

In [3]:
scraper = RTScraper('https://russian.rt.com/rss')
news_df = scraper.get_latest_news()

In [5]:
news_df

,title,link,publication_time,text
0,Нетаньяху: война с Израилем станет для «Хезбол...,https://russian.rt.com/world/news/1229768-neta...,2023-11-12 00:33:21+03:00,Израильский премьер Биньямин Нетаньяху заявил ...
1,Фанаты устроили массовую драку на матче чемпио...,https://russian.rt.com/sport/news/1229769-fana...,2023-11-12 00:25:49+03:00,Матч 34го тура чемпионата Бразилии по футболу ...
2,BFMTV: более 16 тысяч человек вышли на пропале...,https://russian.rt.com/world/news/1229767-demo...,2023-11-12 00:19:20+03:00,Свыше 16 тыс человек вышли на пропалестинскую ...
3,Голевая передача Костина помогла «Детройту» об...,https://russian.rt.com/sport/news/1229766-kost...,2023-11-12 00:17:50+03:00,Детройт Ред Уингз на своём льду нанёс поражени...
4,Хельвани: Усмана Нурмагомедова дисквалифициров...,https://russian.rt.com/sport/news/1229765-usma...,2023-11-12 00:08:39+03:00,Атлетическая комиссия штата Калифорния CSAC от...
5,"Янчук считает, что Шевченко может добиться бол...",https://russian.rt.com/sport/news/1229764-yanc...,2023-11-12 00:04:51+03:00,Заслуженный тренер России Виктор Янчук признал...
6,Аэропорты Внуково и Домодедово вводили огранич...,https://russian.rt.com/russia/news/1229763-ogr...,2023-11-12 00:02:00+03:00,Московские аэропорты Внуково и Домодедово ввод...
7,«Всё больше разногласий»: как Киев пытается до...,https://russian.rt.com/world/article/1229679-u...,2023-11-12 00:00:47+03:00,Руководитель офиса Владимира Зеленского Андрей...
8,"Добивание Сперцяна, гол Кассьерры и штанга Мос...",https://russian.rt.com/sport/article/1229681-z...,2023-11-11 23:54:10+03:00,Зенит и Краснодар разошлись миром в центрально...
9,Самолёт ВВС США потерпел крушение в Средиземно...,https://russian.rt.com/world/news/1229761-krus...,2023-11-11 23:53:26+03:00,Самолёт американских ВВС потерпел крушение в С...
